# Exemplo Análise de textos - movie review

O arquivo *movie_review* armazena uma coleção de avaliações com os respectivos sentimentos, sendo 1 - positivo e 0 - negativo.

O arquivo foi baseado neste dataset: http://ai.stanford.edu/~amaas/data/sentiment/

## Carregar pacotes

In [1]:
library(tidyverse)
library(magrittr)
library(tm)
library(SnowballC)
library(wordcloud)
library(text2vec)
library(glmnet)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.0.0     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ dplyr   0.7.6
✔ tidyr   0.8.1     ✔ stringr 1.3.1
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘magrittr’

The following object is masked from ‘package:purrr’:

    set_names

The following object is masked from ‘package:tidyr’:

    extract

Loading required package: NLP

Attaching package: ‘NLP’

The following object is masked from ‘package:ggplot2’:

    annotate

Loading required package: RColorBrewer
Loading required package: Matrix

Attaching package: ‘Matrix’

The following object is masked from ‘package:tidyr’:

    expand

Loading required package: foreach

Attaching package: ‘foreach’

The following objects are masked from ‘package:purrr’:

    accumulate, when

Loaded glmn

## Carregar dados (*Corpus*)

In [2]:
data("movie_review")

In [3]:
movie_review %>% 
    head()

id,sentiment,review
5814_8,1,"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally starts is only on for 20 minutes or so excluding the Smooth Criminal sequence and Joe Pesci is convincing as a psychopathic all powerful drug lord. Why he wants MJ dead so bad is beyond me. Because MJ overheard his plans? Nah, Joe Pesci's character ranted that he wanted people to know it is he who is supplying drugs etc so i dunno, maybe he just hates MJ's music.<br /><br />Lots of cool things in this like MJ turning into a car and a robot and the whole Speed Demon sequence. Also, the director must have had the patience of a saint when it came to filming the kiddy Bad sequence as usually directors hate working with one kid let alone a whole bunch of them performing a complex dance scene.<br /><br />Bottom line, this movie is for people who like MJ on one level or another (which i think is most people). If not, then stay away. It does try and give off a wholesome message and ironically MJ's bestest buddy in this movie is a girl! Michael Jackson is truly one of the most talented people ever to grace this planet but is he guilty? Well, with all the attention i've gave this subject....hmmm well i don't know because people can be different behind closed doors, i know this for a fact. He is either an extremely nice but stupid guy or one of the most sickest liars. I hope he is not the latter."
2381_9,1,"\""The Classic War of the Worlds\"" by Timothy Hines is a very entertaining film that obviously goes to great effort and lengths to faithfully recreate H. G. Wells' classic book. Mr. Hines succeeds in doing so. I, and those who watched his film with me, appreciated the fact that it was not the standard, predictable Hollywood fare that comes out every year, e.g. the Spielberg version with Tom Cruise that had only the slightest resemblance to the book. Obviously, everyone looks for different things in a movie. Those who envision themselves as amateur \""critics\"" look only to criticize everything they can. Others rate a movie on more important bases,like being entertained, which is why most people never agree with the \""critics\"". We enjoyed the effort Mr. Hines put into being faithful to H.G. Wells' classic novel, and we found it to be very entertaining. This made it easy to overlook what the \""critics\"" perceive to be its shortcomings."
7759_3,0,"The film starts with a manager (Nicholas Bell) giving welcome investors (Robert Carradine) to Primal Park . A secret project mutating a primal animal using fossilized DNA, like Jurassik Park, and some scientists resurrect one of nature's most fearsome predators, the Sabretooth tiger or Smilodon . Scientific ambition turns deadly, however, and when the high voltage fence is opened the creature escape and begins savagely stalking its prey - the human visitors , tourists and scientific.Meanwhile some youngsters enter in the restricted area of the security center and are attacked by a pack of large pre-historical animals which are deadlier and bigger . In addition , a security agent (S

## Pré processamento

As funções de expressão regulares auxiliam na tarefa de pré processamento.

Alguns exemplos são encontrados aqui: https://stat.ethz.ch/R-manual/R-devel/library/base/html/regex.html

In [4]:
texto_preprocessado <- movie_review$review %>% 
    gsub('[[:cntrl:]]', "", .) %>% # retira caracteres de controle
    gsub("http\\S+\\s*", "", .) %>% # retira caracteres relacionados ao html
    gsub("\\d+", "", .) %>% # retira caracteres numericos 
    gsub("[^[:graph:]]", " ", .) %>% # retira caracteres gráficos
    str_replace_all("[^[:alnum:]]", " ") %>% # retira símbolos não alfanuméricos
    str_replace_all("\\s+", " ") %>% # retira múltiplos espaços em branco
    str_to_lower() # texto para minusculo

In [5]:
texto_preprocessado %>% 
    head()

[1] "with all this stuff going down at the moment with mj i ve started listening to his music watching the odd documentary here and there watched the wiz and watched moonwalker again maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent moonwalker is part biography part feature film which i remember going to see at the cinema when it was originally released some of it has subtle messages about mj s feeling towards the press and also the obvious message of drugs are bad m kay br br visually impressive but of course this is all about michael jackson so unless you remotely like mj in anyway then you are going to hate this and find it boring some may call mj an egotist for consenting to the making of this movie but mj and most of his fans would say that he made it for the fans which if true is really nice of him br br the actual feature film bit when it finally starts is only on for minutes or so excluding the smooth criminal sequence and joe pesci is convincing as a psychopathic all powerful drug lord why he wants mj dead so bad is beyond me because mj overheard his plans nah joe pesci s character ranted that he wanted people to know it is he who is supplying drugs etc so i dunno maybe he just hates mj s music br br lots of cool things in this like mj turning into a car and a robot and the whole speed demon sequence also the director must have had the patience of a saint when it came to filming the kiddy bad sequence as usually directors hate working with one kid let alone a whole bunch of them performing a complex dance scene br br bottom line this movie is for people who like mj on one level or another which i think is most people if not then stay away it does try and give off a wholesome message and ironically mj s bestest buddy in this movie is a girl michael jackson is truly one of the most talented people ever to grace this planet but is he guilty well with all the attention i ve gave this subject hmmm well i don t know because people can be different behind closed doors i know this for a fact he is either an extremely nice but stupid guy or one of the most sickest liars i hope he is not the latter "                                                                                                                 
[2] " the classic war of the worlds by timothy hines is a very entertaining film that obviously goes to great effort and lengths to faithfully recreate h g wells classic book mr hines succeeds in doing so i and those who watched his film with me appreciated the fact that it was not the standard predictable hollywood fare that comes out every year e g the spielberg version with tom cruise that had only the slightest resemblance to the book obviously everyone looks for different things in a movie those who envision themselves as amateur critics look only to criticize everything they can others rate a movie on more important bases like being entertained which is why most people never agree with the critics we enjoyed the effort mr hines put into being faithful to h g wells classic novel and we found it to be very entertaining this made it easy to overlook what the critics perceive to be its shortcomings "                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

Para que este processo não seja tão repetitivo, pode-se criar funções que podem chamadas quando necessário.

In [11]:
preprocessamento_texto <- function(texto) {
  
    texto %>% 
        gsub('[[:cntrl:]]', "", .) %>% # retira caracteres de controle
        gsub("http\\S+\\s*", "", .) %>% # retira caracteres relacionados ao html
        gsub("\\d+", "", .) %>% # retira caracteres numericos 
        gsub("[^[:graph:]]", " ", .) %>% # retira caracteres gráficos
        str_replace_all("[^[:alnum:]]", " ") %>% # retira símbolos não alfanuméricos
        str_replace_all("\\s+", " ") %>% # retira múltiplos espaços em branco
        str_to_lower() # texto para minusculo
    
}

## Tabela de sinônimos

A tabela de sinônimos é uma etapa importante, porém ela pode ser desenvolvida durante todo o projeto de análise de texto.

## Tokens

Nesta etapa ocorre a divisão do conjunto de caracteres em uma lista de termos:

In [6]:
# exemplo
stringr::str_split( texto_preprocessado[1], 
                   pattern = stringr::boundary("word") )

[[1]]
  [1] "with"         "all"          "this"         "stuff"        "going"       
  [6] "down"         "at"           "the"          "moment"       "with"        
 [11] "mj"           "i"            "ve"           "started"      "listening"   
 [16] "to"           "his"          "music"        "watching"     "the"         
 [21] "odd"          "documentary"  "here"         "and"          "there"       
 [26] "watched"      "the"          "wiz"          "and"          "watched"     
 [31] "moonwalker"   "again"        "maybe"        "i"            "just"        
 [36] "want"         "to"           "get"          "a"            "certain"     
 [41] "insight"      "into"         "this"         "guy"          "who"         
 [46] "i"            "thought"      "was"          "really"       "cool"        
 [51] "in"           "the"          "eighties"     "just"         "to"          
 [56] "maybe"        "make"         "up"           "my"           "mind"        
 [61] "whether"      "he"           "is"           "guilty"       "or"          
 [66] "innocent"     "moonwalker"   "is"           "part"         "biography"   
 [71] "part"         "feature"      "film"         "which"        "i"           
 [76] "remember"     "going"        "to"           "see"          "at"          
 [81] "the"          "cinema"       "when"         "it"           "was"         
 [86] "originally"   "released"     "some"         "of"           "it"          
 [91] "has"          "subtle"       "messages"     "about"        "mj"          
 [96] "s"            "feeling"      "towards"      "the"          "press"       
[101] "and"          "also"         "the"          "obvious"      "message"     
[106] "of"           "drugs"        "are"          "bad"          "m"           
[111] "kay"          "br"           "br"           "visually"     "impressive"  
[116] "but"          "of"           "course"       "this"         "is"          
[121] "all"          "about"        "michael"      "jackson"      "so"          
[126] "unless"       "you"          "remotely"     "like"         "mj"          
[131] "in"           "anyway"       "then"         "you"          "are"         
[136] "going"        "to"           "hate"         "this"         "and"         
[141] "find"         "it"           "boring"       "some"         "may"         
[146] "call"         "mj"           "an"           "egotist"      "for"         
[151] "consenting"   "to"           "the"          "making"       "of"          
[156] "this"         "movie"        "but"          "mj"           "and"         
[161] "most"         "of"           "his"          "fans"         "would"       
[166] "say"          "that"         "he"           "made"         "it"          
[171] "for"          "the"          "fans"         "which"        "if"          
[176] "true"         "is"           "really"       "nice"         "of"          
[181] "him"          "br"           "br"           "the"          "actual"      
[186] "feature"      "film"         "bit"          "when"         "it"          
[191] "finally"      "starts"       "is"           "only"         "on"          
[196] "for"          "minutes"      "or"           "so"           "excluding"   
[201] "the"          "smooth"       "criminal"     "sequence"     "and"         
[206] "joe"          "pesci"        "is"           "convincing"   "as"          
[211] "a"            "psychopathic" "all"          "powerful"     "drug"        
[216] "lord"         "why"          "he"           "wants"        "mj"          
[221] "dead"         "so"           "bad"          "is"           "beyond"      
[226] "me"           "because"      "mj"           "overheard"    "his"         
[231] "plans"        "nah"          "joe"          "pesci"        "s"           
[236] "character"    "ranted"       "that"         "he"           "wanted"      
[241] "people"       "to"           "know"         "it"           "is"          
[246] "he"           "who

Também pode-se criar funções que podem chamadas quando necessário.

In [14]:
preprocessamento_token <- function( texto ) {
  
  stringr::str_split( texto, pattern = stringr::boundary("word") )
  
}

## Matriz de documentos e termos

Neste exemplo iremos criar uma matriz pelo método *Bag of Words* (TF-DF), dado que desejamos criar uma nuvem de palavras. Na etapa de desenvolver o modelo de sentimento serão apresentados os métodos *one hot encoding* e TF-IDF.

O processo de criar a matriz é muito oneroso computacionalmente, tanto de processamento quanto de memória. Desta forma, iremos utilizar algumas funções do pacote *text2vec* (http://text2vec.org) que otimiza este trabalho.

In [15]:
# criamos uma função de iteração
iterador <- itoken( movie_review$review, # textos
                    preprocessor = preprocessamento_texto, # funcao de pre processamento
                    tokenizer = preprocessamento_token, # divisao dos termos
                    ids = movie_review$id, # id do texto
                    progressbar = FALSE )

A matriz precisa ser construída a partir de um vocabulário

In [18]:
vocabulario <- create_vocabulary( it = iterador, 
                                  ngram = c(1, 1), # combinacao de palavras
                                  stopwords = stopwords("english") # palavras comuns que podem ser desconsideradas 
                                )

In [ ]:
# stopwords
stopwords("english")

In [ ]:
vocabulario

Como pode ser visto, muitos termos não fazem sentido estar nas análises. Portanto, torna-se importante podar o vocabulário.

In [20]:
vocabulario <- prune_vocabulary( vocabulario, 
                                 term_count_min = 10,  
                                 doc_proportion_min = 0.05, 
                                 doc_proportion_max = 0.9 )

In [ ]:
vocabulario

Também é observados termos com somente uma ou duas letras/símbolos. Desta forma, também é importante ajustar estas ocorrências.

In [23]:
vocabulario %<>% 
    filter( nchar(term) >= 4 )

In [28]:
vocabulario

term,term_count,doc_count
overall,268,250
based,275,253
direction,275,255
waste,278,251
unfortunately,279,255
couldn,281,259
entertaining,282,257
already,283,254
lives,283,250
finally,289,263


Após estas etapas, a matriz de termos pode ser construída.

In [25]:
document_term_matrix <- create_dtm( iterador, 
                                    vocab_vectorizer( vocabulario ) )

In [26]:
dim(document_term_matrix)

[1] 5000  281

In [ ]:
document_term_matrix

## Análises

### Wordcloud

Utilizando-se o vocabulário criado anteriormente, podemos criar a nuvem de palavras.

In [ ]:
# 50 termos mais frequentes
vocabulario %>% 
    arrange( desc(term_count) )  %>% 
    head( 50 )

In [ ]:
wordcloud( words = vocabulario$term, 
           freq = vocabulario$term_count, 
           max.words =  50,
           colors = c("blue","red") )

### Análise de sentimentos

Uma forma de realilzar uma análise de sentimentos é aplicar um *Machine Learning* supervisionado utilizando a matriz de documentos e termos. No caso do nosso exemplo *movie reviews*, podemos aplicar um modelo de regressão logística, uma vez que a classificação é binária ( 1 - positivo, 0 - negativo).

In [35]:
movie_review %>% 
    count( sentiment )

sentiment,n
0,2483
1,2517


Vimos anteriormente que a matriz de documentos e termos foi construída utilizando o método *Bag of words* e armazenada no objeto *document_term_matrix*. Iremos criar outras duas matrizes utilizando os métodos *one hot enconding* e TF-IDF.

- TF-IDF: *Inverse Document Frequency*

In [38]:
# gera o método de transformação TF-IDF
tfidf = TfIdf$new()
# transforma a matriz
document_term_matrix_idf <- fit_transform(document_term_matrix, tfidf)

In [ ]:
document_term_matrix_idf

- *One hot encoding*

In [65]:
# transforma a matriz
document_term_matrix_onehot <- (document_term_matrix > 0) * 1

In [ ]:
document_term_matrix_onehot

Após definirmos as matrizes, podemos aplicar os modelos de regressão logística. Como são muitos termos, não é recomendado verificar os coeficientes.

In [71]:
model_onehot <- glmnet( y = movie_review$sentiment, 
       x = document_term_matrix_onehot, 
       family = 'binomial')

In [72]:
model_tfidf <- glmnet( y = movie_review$sentiment, 
       x = document_term_matrix_idf, 
       family = 'binomial')

In [73]:
model_bagwords <- glmnet( y = movie_review$sentiment, 
       x = document_term_matrix, 
       family = 'binomial')

- Podemos avaliar os modelos

In [86]:
pred_model_onehot <- predict( model_onehot, s=0.01, document_term_matrix_onehot, type = 'response' )
pred_model_tfidf <- predict( model_tfidf, s=0.01, document_term_matrix_idf, type = 'response')
pred_model_bagwords <- predict( model_bagwords, s=0.01, document_term_matrix, type = 'response')

In [87]:
# metodo onehot
InformationValue::confusionMatrix( movie_review$sentiment, 
                                   pred_model_onehot,
                                   threshold = 0.5 )

,0,1
0,1918,401
1,565,2116


In [88]:
# metodo tf-idf
InformationValue::confusionMatrix( movie_review$sentiment, 
                                   pred_model_tfidf,
                                   threshold = 0.5 )

,0,1
0,1961,387
1,522,2130


In [89]:
# metodo bag of words
InformationValue::confusionMatrix( movie_review$sentiment, 
                                   pred_model_bagwords,
                                   threshold = 0.5 )

,0,1
0,1906,363
1,577,2154
